In [1]:
# Import a Kalman filter and other useful libraries
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import poly1d

from datetime import datetime



In [20]:
df_cie=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\logo_all.csv",)
df_cie=df_cie
logo=df_cie['Logo'].tolist()

In [21]:
### Price ###
#Data=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_logreturn_total.csv",index_col='Date')

###Std
#Data=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_std25_total.csv",index_col='Date')

#Log Return
Data=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_logreturn_total.csv",index_col='Date')

In [22]:
#Pour autre que STD
#Data=Data.drop(columns='APTV') #faire le cleanage dans la recolte

# pour std

Data=Data[35:].drop(columns='APTV').dropna() #Colonnes APTV que des NaN et les premiere data sont NaN car servent à calculer la std

Data.head(1)


,AAPL,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,APA,AIV,AMAT,ADM,ARNC,AJG,AIZ,ATO,T,ADSK
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-25,-0.025906,-0.001361,-0.001476,-0.004116,-0.012388,-0.002756,-0.003617,0.005008,-0.003221,-0.00096,...,0.013656,-0.006119,-0.010305,-0.01137,-0.048155,-0.006745,0.008523,-0.003381,0.004688,-0.000967


# Kalman

In [23]:
logo_all=logo.copy()
logo_all.remove('AAPL')

logo_Kalman=['AAPL_Kalman']
for l in logo:
    logo_Kalman.append(l+'_Kalman')
   

In [24]:
# Construct a Kalman filter
kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = 0,
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)


In [25]:
# Use the observed values of the price to get a rolling mean

df_kelman=pd.DataFrame()


for l in logo:
    try:
        state_means, _ = kf.filter(Data[l])
        state_means = pd.Series(state_means.flatten(), index=Data.index)
        df_kelman[l]=Data[l]
        df_kelman[l+'_Kalman']=state_means
    except:
        pass

In [26]:
df_kelman.tail(3)

,MMM,MMM_Kalman,ABT,ABT_Kalman,ABBV,ABBV_Kalman,ABMD,ABMD_Kalman,ACN,ACN_Kalman,...,APA,APA_Kalman,AIV,AIV_Kalman,AAPL,AAPL_Kalman,AMAT,AMAT_Kalman,ADM,ADM_Kalman
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,-0.020347,-0.004399,-0.008423,-0.004538,0.000174,-0.000862,0.003523,-0.001572,-0.001956,-0.002379,...,-0.122013,-0.014496,-0.011378,0.001919,-0.019764,-0.007578,-0.041972,-0.004633,-0.024582,-0.002365
2016-01-07,-0.024663,-0.006326,-0.024258,-0.006414,-0.002967,-0.001062,-0.040431,-0.005269,-0.029806,-0.004988,...,-0.052756,-0.018136,-0.005738,0.001191,-0.043121,-0.010959,-0.029768,-0.007024,-0.024042,-0.004427
2016-01-08,-0.003411,-0.006049,-0.021166,-0.007817,-0.027647,-0.003591,0.031955,-0.001728,-0.009728,-0.005439,...,0.002732,-0.016151,-0.019197,-0.000748,0.005274,-0.009415,-0.024707,-0.008706,-0.003773,-0.004365


In [56]:
#df_kelman.tail(3).to_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Kalman_All_and_Pred_logreturn_test.csv")#

In [94]:
# Plot original data and estimated mean
#plt.figure(num=None, figsize=(30, 20), dpi=80)
#plt.plot(state_means)
#plt.plot()
#plt.title('Kalman filter estimate of average')
#plt.legend(['Kalman Estimate', 'X'])
#plt.xlabel('Day')
#plt.ylabel('Price');

In [95]:
#plt.figure(num=None, figsize=(30, 20), dpi=80, facecolor='w', edgecolor='k')
#plt.plot(state_means[-200:])
#plt.plot(x['Close'][-200:])
#
#plt.title('Kalman filter estimate of average')
#plt.legend(['Kalman Estimate', 'X'])
#plt.xlabel('Day')
#plt.ylabel('Price');

# ARIMA

In [5]:
from pandas import read_csv
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

In [6]:
#get data
def GetData(fileName):
    return read_csv(fileName, header=0, parse_dates=[0], index_col=0).values

#Function that calls ARIMA model to fit and forecast the data
def StartARIMAForecasting(Actual, P, D, Q):
	model = ARIMA(Actual, order=(P, D, Q))
	model_fit = model.fit(disp=0)
	prediction = model_fit.forecast()[0]
	return prediction
    



In [7]:
#Get exchange rates
ActualData=Data['AAPL']
ActualData=pd.DataFrame(ActualData)
ActualData.columns=['Close']
ActualData.tail()

,Close
Date,
2016-01-04,0.000855
2016-01-05,-0.025379
2016-01-06,-0.019764
2016-01-07,-0.043121
2016-01-08,0.005274


In [27]:
def MNum_Arima(logo):
    ActualData=Data[logo]
    NumberOfElements = len(ActualData)
    TrainingSize = int(NumberOfElements * 0.7)
    TrainingData = ActualData[0:TrainingSize]
    TestData = ActualData[TrainingSize:NumberOfElements]
    
    #new arrays to store actual and predictions
    Actual = [x for x in TrainingData]
    Predictions = list()
    
    
    #in a for loop, predict values using ARIMA model
    for timepoint in range(len(TestData)):
        ActualValue =  TestData[timepoint]
        #forcast value
        Prediction = StartARIMAForecasting(Actual, 3,1,1)    
        #print('Actual=%f, Predicted=%f' % (ActualValue, Prediction))
        #add it in the list
        Predictions.append(Prediction)
        Actual.append(ActualValue)
        
    return [TestData,Predictions]
        
    
    

In [28]:
#Prédiction est une liste d'array à 1 element)

def conversion(Predictions):
    L=[]
    for k in range(len(Predictions)):
        L.append(Predictions[k][0])
    return L

# Forecasting

In [10]:
df_ARIMA_pred=pd.DataFrame()

error=[]
for l in logo:
    try:
        result=MNum_Arima(l)
        df_ARIMA_pred[l+'_ARIMA']=conversion(result[1])
    except:
        error.append(l)
   

C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\tsa\tsatools.py:607: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\tsa\tsatools.py:609: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\tsa\tsatools.py:650: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\tsa\tsatools.py:651: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check 

C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Geoffroy\Anaconda3\envs\tensorflow_env\lib\site-packages\statsmodels\ba

Problème quand ARIMA ne converge pas (SVD did not converge) le programme tourne longtemps avant de s'en rendre compte)

In [11]:
def remove_error(L,L_error):
    if len(error)==0:
        return L
    else:
        for l in L_error:
            L.remove(l)
    return L
    

In [12]:
logo_se=logo.copy()
remove_error(logo_se,error) ;#à faire tourner que une fois

In [13]:
df_ARIMA_pred.tail(3).reset_index(drop=True)

,AAP_ARIMA,AMT_ARIMA,APA_ARIMA,AIV_ARIMA,AAPL_ARIMA
0,-0.004489,0.002389,-0.001601,0.003075,-0.000381
1,-0.002434,0.000990,-0.010194,-0.000128,-0.000619
2,0.002325,-0.002767,-0.003833,0.000403,0.001207


In [14]:
Data.reset_index().tail()

,Date,AAPL,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,...,APA,AIV,AMAT,ADM,ARNC,AJG,AIZ,ATO,T,ADSK
216,2016-01-04,0.000855,-0.025686,-0.045090,-0.027901,-0.057445,-0.025882,-0.028562,-0.021194,-0.035465,...,-0.000900,-0.009538,-0.010770,-0.023167,-0.016344,-0.018241,-0.025401,-0.010685,-0.001745,-0.010228
217,2016-01-05,-0.025379,0.004350,-0.000233,-0.004175,-0.002820,0.005191,-0.012841,0.004015,-0.007246,...,-0.020465,0.025154,0.001082,0.011099,-0.046373,0.002733,0.016546,0.003681,0.006963,0.001326
218,2016-01-06,-0.019764,-0.020347,-0.008423,0.000174,0.003523,-0.001956,-0.009468,-0.014398,-0.093312,...,-0.122013,-0.011378,-0.041972,-0.024582,-0.073859,-0.019540,-0.013240,0.003190,-0.015441,-0.014175
219,2016-01-07,-0.043121,-0.024663,-0.024258,-0.002967,-0.040431,-0.029806,-0.014235,-0.021208,-0.096309,...,-0.052756,-0.005738,-0.029768,-0.024042,-0.040290,-0.020962,-0.022724,-0.009761,-0.016280,-0.061654
220,2016-01-08,0.005274,-0.003411,-0.021166,-0.027647,0.031955,-0.009728,-0.015560,-0.014241,-0.061174,...,0.002732,-0.019197,-0.024707,-0.003773,-0.024481,-0.010648,-0.012281,0.001125,0.000895,-0.016571


In [15]:
df_ARIMA_all=pd.DataFrame()
Data=Data.tail(3).reset_index()
Arima=df_ARIMA_pred.tail(3).reset_index(drop=True)

df_ARIMA_all['Date']=Data['Date']

for l in logo_se:
    df_ARIMA_all[l]=Data[l]
    df_ARIMA_all[l+'_ARIMA']=Arima[l+'_ARIMA']
    
df_ARIMA_all=df_ARIMA_all.set_index('Date')


In [16]:
df_ARIMA_all

,AAP,AAP_ARIMA,AMT,AMT_ARIMA,APA,APA_ARIMA,AIV,AIV_ARIMA,AAPL,AAPL_ARIMA
Date,,,,,,,,,,
2016-01-06,-0.026811,-0.004489,-0.003556,0.002389,-0.122013,-0.001601,-0.011378,0.003075,-0.019764,-0.000381
2016-01-07,0.011013,-0.002434,-0.025250,0.000990,-0.052756,-0.010194,-0.005738,-0.000128,-0.043121,-0.000619
2016-01-08,-0.022216,0.002325,-0.016628,-0.002767,0.002732,-0.003833,-0.019197,0.000403,0.005274,0.001207


In [17]:
df_ARIMA_all.to_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\ARIMA_All_and_Pred_logreturn_test.csv")


In [31]:
df_ARIMA_all

,AAP,AAP_ARIMA,AMT,AMT_ARIMA,APA,APA_ARIMA,AIV,AIV_ARIMA,AAPL,AAPL_ARIMA
Date,,,,,,,,,,
2016-01-06,-0.026811,-0.004489,-0.003556,0.002389,-0.122013,-0.001601,-0.011378,0.003075,-0.019764,-0.000381
2016-01-07,0.011013,-0.002434,-0.025250,0.000990,-0.052756,-0.010194,-0.005738,-0.000128,-0.043121,-0.000619
2016-01-08,-0.022216,0.002325,-0.016628,-0.002767,0.002732,-0.003833,-0.019197,0.000403,0.005274,0.001207


In [35]:
k='ARE'
res=MNum_Arima(l)

ValueError: The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.

In [37]:
Data

,AAPL,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,APA,AIV,AMAT,ADM,ARNC,AJG,AIZ,ATO,T,ADSK
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-25,-0.025906,-0.001361,-0.001476,-0.004116,-0.012388,-0.002756,-0.003617,0.005008,-0.003221,-0.000960,...,0.013656,-0.006119,-0.010305,-0.011370,-0.048155,-0.006745,0.008523,-0.003381,0.004688,-0.000967
2015-02-26,0.012539,0.004254,0.004421,-0.001651,0.013700,-0.003317,-0.004487,0.011336,-0.006473,-0.005584,...,-0.031297,-0.009924,-0.007599,-0.011500,0.007307,0.000846,-0.003433,-0.012495,0.008441,-0.016258
2015-02-27,-0.015104,-0.005676,-0.005054,-0.000331,-0.003449,-0.002995,-0.001286,0.001771,0.009693,-0.002900,...,0.001520,0.015512,0.005604,0.006916,-0.021405,-0.006998,0.003433,0.010424,0.001738,0.051588
2015-03-02,0.004892,0.010910,-0.003172,-0.000661,0.022769,0.012693,-0.003221,0.008685,0.031648,-0.001033,...,-0.000760,0.006877,0.022890,-0.000627,0.010760,-0.000852,0.006832,-0.017498,0.002312,-0.003743
2015-03-03,0.002089,-0.013165,-0.002757,-0.013991,-0.033353,-0.003736,-0.006257,-0.010077,-0.022048,-0.001940,...,0.014037,-0.006611,-0.045703,-0.006920,0.015267,-0.002986,-0.005201,0.022200,-0.006662,-0.020522
2015-03-04,-0.006359,-0.006618,-0.001700,0.010843,0.017959,0.000770,-0.007387,-0.017367,-0.029081,-0.023847,...,-0.002852,-0.007724,-0.008820,-0.009302,-0.039642,-0.005785,-0.013452,-0.003195,-0.011987,-0.002875
2015-03-05,-0.016710,0.002450,0.009105,-0.058242,0.002283,0.012462,0.015578,0.012672,-0.023218,0.011142,...,-0.016672,0.009315,-0.003715,0.008460,-0.014498,-0.001720,0.017842,0.005819,0.000000,0.006854
2015-03-06,0.001502,-0.019342,-0.020013,-0.021690,-0.016923,-0.015105,-0.009491,-0.013703,-0.020340,-0.022880,...,-0.035785,-0.023859,-0.011229,-0.026247,0.006930,-0.014089,-0.000811,-0.023098,-0.015412,-0.019409
2015-03-09,0.004256,0.012155,0.007072,-0.001799,-0.007318,0.001323,0.010564,0.004888,0.003419,-0.005246,...,-0.027623,0.004601,0.007500,0.003022,-0.055373,0.005877,0.009855,-0.000575,-0.003890,0.000324
